In [6]:

import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence


In [7]:

# Label parser that supports multiple tumors
def parse_label(label_path):
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()
            tumor_coords = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                presence = int(parts[0])
                if presence == 1:
                    coords = list(map(float, parts[1:]))
                    tumor_coords.append(coords)
            if len(tumor_coords) == 0:
                return 0, [None, None, None, None]
            # For now, use the first tumor for training (can extend to multiple if needed)
            return 1, tumor_coords[0]
    except Exception as e:
        return 0, [None, None, None, None]


In [8]:

class TumorDataGenerator(Sequence):
    def __init__(self, df, batch_size=16, img_size=224, shuffle=True):
        self.df = df.reset_index(drop=True)
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_df = self.df.iloc[index * self.batch_size:(index + 1) * self.batch_size]
        images, labels_class, labels_bbox = [], [], []
        for _, row in batch_df.iterrows():
            image = cv2.imread(row['Image_Path'])
            image = cv2.resize(image, (self.img_size, self.img_size))
            image = image / 255.0
            presence, coords = parse_label(row['Label_Path'])
            images.append(image)
            labels_class.append(presence)
            if presence == 1:
                labels_bbox.append(coords)
            else:
                labels_bbox.append([0, 0, 0, 0])
        return np.array(images), {'class_output': np.array(labels_class), 'bbox_output': np.array(labels_bbox)}

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [9]:

def build_model(img_size=224):
    base_model = ResNet50(include_top=False, input_shape=(img_size, img_size, 3), weights='imagenet')
    x = GlobalAveragePooling2D()(base_model.output)
    class_output = Dense(1, activation='sigmoid', name='class_output')(x)
    bbox_output = Dense(4, activation='sigmoid', name='bbox_output')(x)
    model = Model(inputs=base_model.input, outputs=[class_output, bbox_output])
    model.compile(optimizer=Adam(),
                  loss={'class_output': 'binary_crossentropy', 'bbox_output': 'mse'},
                  loss_weights={'class_output': 1.0, 'bbox_output': 1.0})
    return model


In [ ]:

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1
for train_idx, val_idx in kf.split(df_train):
    print(f'Fold {fold}')
    train_df = df_train.iloc[train_idx]
    val_df = df_train.iloc[val_idx]
    
    train_gen = TumorDataGenerator(train_df)
    val_gen = TumorDataGenerator(val_df, shuffle=False)

    model = build_model()
    model.fit(train_gen, validation_data=val_gen, epochs=2)

    fold += 1


Fold 1
Epoch 1/2
 88/211 ━━━━━━━━━━━━━━━━━━━━ 7:49 4s/step - bbox_output_loss: 0.0684 - class_output_loss: 0.9480 - loss: 1.0164

In [ ]:

# Load test images only and make predictions
test_images = []
image_ids = []

for _, row in df_test.iterrows():
    image = cv2.imread(row['Image_Path'])
    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    test_images.append(image)
    image_ids.append(row['Image_ID'])

test_images = np.array(test_images)

preds = model.predict(test_images, batch_size=16)
pred_classes = (preds[0] > 0.5).astype(int)
pred_bboxes = preds[1]

# Save results with nulls for class 0
final_predictions = []
for cls, bbox in zip(pred_classes, pred_bboxes):
    if cls[0] == 0:
        final_predictions.append([0, None, None, None, None])
    else:
        final_predictions.append([1] + bbox.tolist())

final_df = pd.DataFrame(final_predictions, columns=['Tumor_Present', 'X_center', 'Y_center', 'Width', 'Height'])
final_df['Image_ID'] = image_ids
final_df.to_csv('test_predictions.csv', index=False)

print("Saved predictions to 'test_predictions.csv'")


In [ ]:

# Example: if you had ground-truth presence labels in df_test
# You could evaluate as follows:

if 'Label_Path' in df_test.columns:
    y_true = []
    for path in df_test['Label_Path']:
        presence, _ = parse_label(path)
        y_true.append(presence)

    y_pred = [x[0] for x in final_predictions]

    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
